In [3]:
import pandas as pd 
grouped_bbq = pd.read_pickle("grouped_bbq.pkl")
#one hot encode bbq type to see what bbqs are available at each group
grouped_bbq = pd.get_dummies(grouped_bbq,prefix=['bbq_type'], columns = ['bbq_type'], drop_first=False)

def create_group_id(df):
    if df['group'] >=0: return str(df['group'])
    else: return df['asset_id']
grouped_bbq["group_id"] = grouped_bbq.apply(create_group_id,axis=1)
grouped_bbq["count"] = 1 


In [4]:
#split multi and single bbqs
grouped_bbq_sum = grouped_bbq[["group_id","bbq_type_Electric","bbq_type_Gas","bbq_type_Wood","count"]].groupby("group_id", as_index=False).agg(["sum"])
grouped_bbq_avg = grouped_bbq[["group_id","latitude","longitude"]].groupby("group_id", as_index=False).agg(["mean"])


In [5]:
amenities = {"bbq": {"csv":'https://www.data.act.gov.au/resource/n3b4-mm52.csv', "colour":"red","name": "BBQ","id":"asset_id","attrib":"bbq_type"} , 
             "drink": {"csv":'https://www.data.act.gov.au/resource/8eg4-uskm.csv', "colour":"blue","name": "Bubbler","id":"id","attrib":""}, 
             "furn": {"csv":'https://www.data.act.gov.au/resource/ch39-bukk.csv', "colour":"orange","name": "Furniture","id":"asset_id","attrib":"feature_type"}, 
             "toilet": {"csv":'https://www.data.act.gov.au/resource/3tyf-txjn.csv', "colour":"brown","name": "Toilet","id":"asset_id","attrib":"toilet_type_text"} ,
             "fitness": {"csv":'https://www.data.act.gov.au/resource/h4qc-3txc.csv', "colour":"grey","name": "Fitness Equipment","id":"id","attrib":"type"} ,
             "playground": {"csv":'https://www.data.act.gov.au/resource/fwth-mr9q.csv', "colour":"yellow","name": "Playground","id":"asset_id","attrib":""} 
            }
for a in amenities:
    amenities[a]["df"] = pd.read_pickle(a+".pkl")

In [6]:
exclude = ["bbq","fitness"]
amenities_near = pd.DataFrame()
for a in amenities:
    if a not in exclude:        
        amenities_near= amenities_near.append(amenities[a]["df"].rename(columns={amenities[a]["id"]:"amen_id"})[["amen_id","cohort","latitude","longitude","x","y"]])
amenities_near = amenities_near.reset_index()


In [7]:
#cross join 
import numpy as np 
def cartesian_product_simplified(left, right):
    la, lb = len(left), len(right)
    ia2, ib2 = np.broadcast_arrays(*np.ogrid[:la,:lb])

    
    return pd.DataFrame(
        np.column_stack([left.values[ia2.ravel()], right.values[ib2.ravel()]]),
        columns=list(left.columns)+ list(right.columns)
        )


bbq_distance =\
cartesian_product_simplified(\
grouped_bbq.rename(columns={"latitude":"lat1","longitude":"lon1"})[["group_id","lat1","lon1"]]\
,amenities_near.rename(columns={"latitude":"lat2","longitude":"lon2"})[["amen_id","cohort","lat2","lon2"]])

bbq_distance = bbq_distance.reset_index()

from math import radians, cos, sin, asin, sqrt

def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    #print(lon1, lat1, lon2, lat2)
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

from haversine import haversine

#bbq_distance = bbq_distance.drop(columns=["index"])

bbq_distance["distance"] = haversine_np(bbq_distance['lon1'].astype(float),bbq_distance['lat1'].astype(float),bbq_distance['lon2'].astype(float),bbq_distance['lat2'].astype(float))* 1000

In [8]:
#get all amenities close to BBQ groups
bbq_amenity = bbq_distance[bbq_distance["distance"]<=100][["group_id","cohort"]]
bbq_amenity = pd.get_dummies(bbq_amenity,prefix=['n'], columns = ['cohort'], drop_first=False)
bbq_amenity_sum = bbq_amenity.groupby("group_id", as_index=False).agg(["sum"])


In [199]:
#print(bbq_amenity["group_id"].unique())

In [9]:
bbq_amen_2  = grouped_bbq_sum.join(bbq_amenity_sum, on=["group_id"])
bbq_amen_2 = bbq_amen_2.join(grouped_bbq_avg, on=["group_id"])
bbq_amen_2.update(bbq_amen_2[[ "n_drink", "n_furn", "n_playground", "n_toilet"]].fillna(0))
bbq_amen_2["amen_count"] = bbq_amen_2[ ["n_drink", "n_furn", "n_playground", "n_toilet"]].sum(axis=1)
#Bokeh maps are in mercator. Convert lat lon fields to mercator units for plotting

def wgs84_to_web_mercator(df, lon, lat):
    """Converts decimal longitude/latitude to Web Mercator format"""
    k = 6378137
    df["x"] = df[lon] * (k * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df


bbq_amen_2=wgs84_to_web_mercator(bbq_amen_2,'longitude','latitude')
bbq_amen_2 = bbq_amen_2.reset_index()


In [13]:
from bokeh.models import *
from bokeh.plotting import *
from bokeh.io import *
from bokeh.tile_providers import *
from bokeh.palettes import *
from bokeh.transform import *
from bokeh.layouts import *

In [136]:
scale=100
x=bbq_amen_2['x']
y=bbq_amen_2['y']

#The range for the map extents is derived from the lat/lon fields. This way the map is automatically centered on the plot elements.
"""
x_min=int(x.mean() - (scale * 150))
x_max=int(x.mean() + (scale * 150))
y_min=int(y.mean() - (scale * 150))
y_max=int(y.mean() + (scale * 150))
"""


16669980 16709980 -4224961 -4192961


In [128]:
#rename to first of each tuple
bbq_amen_2.columns = bbq_amen_2.columns.get_level_values(0)

#add flags, 
bbq_types=['Electric','Gas','Wood']
amen_types=['drink','furn','playground','toilet']
for t in bbq_types:
    bbq_amen_2[t+'_f'] = (bbq_amen_2['bbq_type_'+t]>0).astype('int32')
for a in amen_types:
    bbq_amen_2[a+'_f'] = (bbq_amen_2['n_'+a]>0).astype('int32')
 

In [155]:
x_min, x_max, y_min, y_max = (16579980 , 16619980, -4234961, -4182961)

#Defining the map tiles to use. I use OSM, but you can also use ESRI images or google street maps.

tile_provider=get_provider(Vendors.CARTODBPOSITRON_RETINA)

#Establish the bokeh plot object and add the map tile as an underlay. Hide x and y axis.

plot=figure(
    title='',
    match_aspect=True,
    tools='wheel_zoom,pan,reset,save',
    x_range=(x_min, x_max),
    y_range=(y_min, y_max),
    x_axis_type='mercator',
    y_axis_type='mercator',
    width=800,
    height= 700,
    output_backend="webgl"
    )

plot.grid.visible=True

map=plot.add_tile(tile_provider)
map.level='underlay'

plot.xaxis.visible = False
plot.yaxis.visible=False
plot.title.text_font_size="20px"

plot.toolbar.active_scroll=plot.select_one(WheelZoomTool)

output_notebook()
#function takes a column to determine radius and the dataframe with converted mercator coordinates to create a bubble map. 
def bubble_map(plot,df,radius_col,lon,lat,scale,color='orange',leg_label='Bubble Map'):

  df['radius']=scale
    
  source=ColumnDataSource(df)
  c=plot.circle(x='x',y='y',color=color,source=source,size=1,fill_alpha=0.4,radius='radius',legend_label=leg_label,hover_color='red')

  tip_label='@'+radius_col
  lat_label='@'+lat
  lon_label='@'+lon

  circle_hover = HoverTool(tooltips=[(radius_col,tip_label),('Lat:',lat_label),('Lon:',lon_label)],mode='mouse',point_policy='follow_mouse',renderers=[c])
  circle_hover.renderers.append(c)
  plot.tools.append(circle_hover)

#The legend.click_policy method allows us to toggle layer on/off by clicking the corresponding field in the legend. We'll explore this more later!
  plot.legend.location = "top_right"
  plot.legend.click_policy="hide"

#Create the bubble map. In this case, circle radius is defined by the amount of fatalities. Any column can be chosen to define the radius.
for a in amenities:
    if a not in exclude:        
        '''bubble_map(plot=plot,
                   df=amenities[a]["df"],
                   radius_col='location', 
                   leg_label=amenities[a]["name"],
                   lon='longitude',
                   lat='latitude',
                   color = amenities[a]["colour"],
                   scale=10)
        '''
        pass
    

#add buttons
bbq_type_labels = ["Electric", "Gas", "Wood"]
amen_type_labels = ["Toilet", "Bubbler", "Table", "Playground" ]

heading_div = Div(text="""<h1>Where to BBQ?</h1>""",height=40)

bbq_types_div = Div(text="""<b> <h3>BBQ types </h3></b>""",height=30)
bbq_div = Div(text="""What BBQ types do you want to use?""")
bbq_checkbox_button_group = CheckboxButtonGroup(labels=bbq_type_labels, active=[0, 1 , 2])

amen_types_div = Div(text="""<b><h3> Amenities </h3></b>""",height=30)
amen_div = Div(text="""What amenities do you need near the BBQ? Select all the amenities you require near the BBQ.""")
amen_checkbox_button_group = CheckboxGroup(labels=amen_type_labels, active=[])


data_source = ColumnDataSource(bbq_amen_2)

# This callback triggers the filter when the slider changes

indices = list(range(len(bbq_amen_2)))
filter = IndexFilter(indices)

bbq_filter = CustomJS(args=dict(filter=filter,src=data_source, bbq_buttons = bbq_checkbox_button_group, amen_buttons=amen_checkbox_button_group), code='''

  indices = []
  const bbq_col_names = ['Electric_f','Gas_f','Wood_f']
  const amen_col_names = ["toilet_f", "drink_f","furn_f","playground_f"]
  
  var indicies = [] ;
  for (var i = 0; i < src.get_length(); i++) {
    bbq_col = 0 
    //any type of bbq selected will display
    for (const bbq of bbq_buttons.active) {
      bbq_col = bbq_col + src.data[bbq_col_names[bbq]][i]
    }
    //all amenities have to be available
    amen_col = 1
    for (const amen of amen_buttons.active) {
      amen_col = amen_col * src.data[amen_col_names[amen]][i]
    }
    
    //create index series to pass back and filter dataframe
    if (amen_col> 0 && bbq_col>0) { indices.push(i) }
    
  }
  filter.indices = indices
  src.change.emit()
  
''')

bbq_checkbox_button_group.js_on_click(bbq_filter)
amen_checkbox_button_group.js_on_click(bbq_filter)

view = CDSView(source=data_source, filters=[filter])

c=plot.circle(x='x',y='y',color='red',source=data_source,
              size=10,fill_alpha=0.4,legend_label='BBQs',
              hover_color='red',view=view)

lat_label='@latitude'
lon_label='@longitude'

circle_hover = HoverTool(tooltips=[('Lat:',lat_label),('Lon:',lon_label)],mode='mouse',point_policy='follow_mouse',renderers=[c])
url = " https://www.google.com/maps/search/?api=1&query=@latitude,@longitude"
plot.add_tools(circle_hover, TapTool(callback=OpenURL(url=url)))


#The legend.click_policy method allows us to toggle layer on/off by clicking the corresponding field in the legend. We'll explore this more later!
plot.legend.location = "top_right"
plot.legend.click_policy="hide"

#Create the bubble map. In this case, circle radius is defined by the amount of fatalities. Any column can be chosen to define the radius.

# Make a column layout of widgetbox(slider) and plot, and add it to the current document
layout = column(widgetbox(heading_div),widgetbox(bbq_types_div),widgetbox(bbq_div),widgetbox(bbq_checkbox_button_group),widgetbox(amen_types_div),widgetbox(amen_div),widgetbox(amen_checkbox_button_group),plot)

output_file("./site/index.html")

save(layout)

Loading BokehJS ...

'/home/chez/projects/where_to_bbq/site/index.html'

In [ ]:
print(dir(plot.toolbar))
